In [1]:
from osgeo import gdal, ogr, osr, gdalconst
import os
import datetime 
import numpy as np

In [2]:
start_time = datetime.datetime.now()
folder = 'Yourfolderpath' 
os.chdir(folder) #set default path
for raster in os.listdir(): #loop for every file
    if raster.endswith('tif'):
        inraster = gdal.Open(raster)  #read raster
        prj = osr.SpatialReference()  
        inband = inraster.GetRasterBand(1)  # image band, if it's single band, then it equals 1
        prj.ImportFromWkt(inraster.GetProjection())   #read projection for raster

        outshp = raster[:-4] + ".shp" #name the future shp file
        drv = ogr.GetDriverByName("ESRI Shapefile")
        if os.path.exists(outshp):  #if it exists，delete it and redo.
            drv.DeleteDataSource(outshp)
        Polygon = drv.CreateDataSource(outshp)  #create a target file
        Poly_layer = Polygon.CreateLayer(raster[:-4], srs = prj, geom_type = ogr.wkbLineString) #create a layer for shp file， set linestring
        newField = ogr.FieldDefn('value',ogr.OFTReal)  # to store original raster pixel value
        Poly_layer.CreateField(newField)

        gdal.FPolygonize(inband, None, Poly_layer, 0) #main function for raster to vector
        Polygon.SyncToDisk() 
        Polygon = None
end_time = datetime.datetime.now()
print ("Succeeded at",end_time)
print ("Elapsed Time:",end_time-start_time)  # print time for execution

Succeeded at 2021-12-16 16:04:03.364347
Elapsed Time: 0:00:09.175110
